In [10]:
import os
import pandas as pd
import re

In [ ]:
filefolder = r"D:\SandmanGitCodes\files"  # metadate files saved in local from prior unzip process
out_dir = r"D:\SandmanGitCodes\files\merged"
os.makedirs(out_dir, exist_ok=True)

n = 16
lst_files = []
for i in range(n):
    filename = f"Sandman_Zipped_Metadata_Processing_part{i+1:02d}.xlsx"
    filepath = os.path.join(filefolder,filename)
    lst_files.append(filepath)

# groups in 4 external hard drives, each drive contain multiple partitions from unzip jobs
group_strs = [
    "7&9&14&16",
    "2&10&11&12&13",
    "1&3&8&15",
    "4&5&6",
]

groups_files = [
    [lst_files[int(x.strip()) - 1] for x in s.split("&")]
    for s in group_strs
]

for gi, files in enumerate(groups_files, 1):
    dfs = []
    for f in files:
        df = pd.read_excel(f, engine="openpyxl")
        df["SourceFile"] = os.path.basename(f)
        dfs.append(df)
    merged = pd.concat(dfs, ignore_index=True)
    merged.to_excel(os.path.join(out_dir, f"group{gi}.xlsx"), index=False)
    print(f"Group {gi} done, saved to group{gi}.xlsx")


Group 1 done, saved to group1.xlsx
Group 2 done, saved to group2.xlsx
Group 3 done, saved to group3.xlsx
Group 4 done, saved to group4.xlsx


In [12]:
in_dir = r"D:\SandmanGitCodes\files\merged"
out_dir = r"D:\SandmanGitCodes\files\cleaned"
os.makedirs(out_dir, exist_ok=True)

# loop over 4 source hard drives
for fname in os.listdir(in_dir):
    if fname.lower().endswith(".xlsx") and fname.startswith("group"):
        in_path = os.path.join(in_dir, fname)
        print(f"Processing {in_path} ...")

        # read
        df = pd.read_excel(in_path, engine="openpyxl")

        # ============ cleansing ============
        df["CleanedFileName"] = (
            df["FileName"]
            .str.replace(r"[()]", "", regex=True)
            .str.replace(r"\.zip$", "", regex=True)
            .str.replace(r"^_+", "", regex=True)
            .str.replace("'", "", regex=False)
            )

        # save
        out_path = os.path.join(out_dir, fname)
        df.to_excel(out_path, index=False)
        print(f"Cleaned file saved to {out_path}")

print("All files processed.")


Processing D:\SandmanGitCodes\files\merged\group1.xlsx ...
Cleaned file saved to D:\SandmanGitCodes\files\cleaned\group1.xlsx
Processing D:\SandmanGitCodes\files\merged\group2.xlsx ...
Cleaned file saved to D:\SandmanGitCodes\files\cleaned\group2.xlsx
Processing D:\SandmanGitCodes\files\merged\group3.xlsx ...
Cleaned file saved to D:\SandmanGitCodes\files\cleaned\group3.xlsx
Processing D:\SandmanGitCodes\files\merged\group4.xlsx ...
Cleaned file saved to D:\SandmanGitCodes\files\cleaned\group4.xlsx
All files processed.


In [13]:
in_dir = r"D:\SandmanGitCodes\files\cleaned"
out_dir = r"D:\SandmanGitCodes\files\final"
os.makedirs(out_dir, exist_ok=True)

group_files = [f"group{i}.xlsx" for i in range(1, 5)]

for fname in group_files:
    in_path = os.path.join(in_dir, fname)
    print(f"Processing {in_path} ...")

    df = pd.read_excel(in_path)

    df = df[df["OutputPath"] != "-1"]

    cols_to_keep = ["FullPath", "OutputPath", "CleanedFileName"]
    subset = df[cols_to_keep].copy()

    dedup = subset.drop_duplicates(subset="CleanedFileName", keep="first")

    print(f"{fname}: {len(subset) - len(dedup)} duplicate rows removed")

    out_path = os.path.join(out_dir, fname)
    dedup.to_excel(out_path, index=False)

    print(f"Saved subset to {out_path} with {len(dedup)} rows.")

print("All files processed.")

Processing D:\SandmanGitCodes\files\cleaned\group1.xlsx ...
group1.xlsx: 24 duplicate rows removed
Saved subset to D:\SandmanGitCodes\files\final\group1.xlsx with 3396 rows.
Processing D:\SandmanGitCodes\files\cleaned\group2.xlsx ...
group2.xlsx: 25 duplicate rows removed
Saved subset to D:\SandmanGitCodes\files\final\group2.xlsx with 4967 rows.
Processing D:\SandmanGitCodes\files\cleaned\group3.xlsx ...
group3.xlsx: 409 duplicate rows removed
Saved subset to D:\SandmanGitCodes\files\final\group3.xlsx with 3577 rows.
Processing D:\SandmanGitCodes\files\cleaned\group4.xlsx ...
group4.xlsx: 1 duplicate rows removed
Saved subset to D:\SandmanGitCodes\files\final\group4.xlsx with 2974 rows.
All files processed.


In [14]:
in_dir = r"D:\SandmanGitCodes\files\final"

group_files = [f"group{i}.xlsx" for i in range(1, 5)]

all_dfs = []

for fname in group_files:
    in_path = os.path.join(in_dir, fname)
    print(f"Reading {in_path} ...")

    df = pd.read_excel(in_path)

    df["SourceGroup"] = fname  # "group1.xlsx"

    all_dfs.append(df)

merged_df = pd.concat(all_dfs, ignore_index=True)

print("Merged dataframe shape:", merged_df.shape)

# ========== 0) ==========
need_cols = ["SourceGroup", "CleanedFileName"]
miss = [c for c in need_cols if c not in merged_df.columns]
if miss:
    raise ValueError(f"miss: {miss}")

merged_df = merged_df.sort_values("CleanedFileName", ascending=True, kind="mergesort").reset_index(drop=True)
merged_df["IsDup"] = merged_df["CleanedFileName"].duplicated(keep="first").astype(int)

DATE_TOKEN = re.compile(r'^\d{4}-\d{1,2}-\d{1,2}$')

def is_date_token(tok: str) -> bool:
    return bool(DATE_TOKEN.fullmatch(tok))

def extract_id_from_clean(name: str):
    if pd.isna(name):
        return pd.NA
    s = str(name)
    if s.startswith("_0_0_0"):
        return pd.NA
    parts = s.split("_")
    start = 1 if parts and is_date_token(parts[0]) else 0
    for tok in parts[start:]:
        if tok.isdigit():
            return tok
    return pd.NA

def extract_subject_from_clean(name: str):
    if pd.isna(name):
        return pd.NA
    s = str(name)
    if s.startswith("_0_0_0"):
        return pd.NA
    parts = s.split("_")
    if parts and is_date_token(parts[0]):
        return pd.NA
    for i, tok in enumerate(parts):
        if tok.isdigit():
            subj = "_".join(parts[:i]).strip("_")
            return subj if subj else pd.NA
    return pd.NA

#  Subject / ID / ID_num
if "Subject" not in merged_df.columns:
    merged_df["Subject"] = merged_df["CleanedFileName"].apply(extract_subject_from_clean)
if "ID" not in merged_df.columns:
    merged_df["ID"] = merged_df["CleanedFileName"].apply(extract_id_from_clean)
if "ID_num" not in merged_df.columns:
    merged_df["ID_num"] = pd.to_numeric(merged_df["ID"], errors="coerce")

merged_df["Subject_norm"] = (
    merged_df["Subject"]
      .astype("string")  # None/NA
      .str.lower()
      .str.strip()
)

# 0->A, 1->B, ...
def idx_to_letters(n: int) -> str:
    n = int(n)
    s = ""
    while True:
        s = chr(ord('A') + (n % 26)) + s
        n = n // 26 - 1
        if n < 0:
            return s

# ========== 1) group by name, then by ID ==========
has_name = merged_df["Subject_norm"].notna() & (merged_df["Subject_norm"] != "")

# 1.1
name_sizes = (
    merged_df.loc[has_name]
             .groupby(["SourceGroup", "Subject_norm"], sort=False)["Subject_norm"]
             .transform("size")
)
merged_df["isNameDup"] = 0
merged_df.loc[has_name, "isNameDup"] = (name_sizes > 1).astype(int)

# 1.2
merged_df["NameGroup"] = "A"
to_rank_name_idx = merged_df.index[has_name & (merged_df["isNameDup"] == 1)]
if len(to_rank_name_idx):
    rank_name = (
        merged_df.loc[to_rank_name_idx]
                 .sort_values(["SourceGroup", "Subject_norm", "CleanedFileName"])
                 .groupby(["SourceGroup", "Subject_norm"], sort=False)
                 .cumcount()
                 .map(idx_to_letters)
    )
    merged_df.loc[rank_name.index, "NameGroup"] = rank_name

# 1.3 
merged_df["isIDDup_Name"] = 0
merged_df["NameIDGroup"] = "A"
mask_name_any = has_name
name_id_sizes = (
    merged_df.loc[mask_name_any]
             .groupby(["SourceGroup", "Subject_norm", "ID_num"], sort=False)["ID_num"]
             .transform("size")
)
dup_name_id_idx = merged_df.index[mask_name_any & (name_id_sizes > 1)]
merged_df.loc[dup_name_id_idx, "isIDDup_Name"] = 1
if len(dup_name_id_idx):
    rank_name_id = (
        merged_df.loc[dup_name_id_idx]
                 .sort_values(["SourceGroup", "Subject_norm", "ID_num", "CleanedFileName"])
                 .groupby(["SourceGroup", "Subject_norm", "ID_num"], sort=False)
                 .cumcount()
                 .map(idx_to_letters)
    )
    merged_df.loc[rank_name_id.index, "NameIDGroup"] = rank_name_id

# ========== 2) name na: by ID ==========
no_name = ~has_name & merged_df["ID_num"].notna()
merged_df["isIDDup_NoName"] = 0
merged_df["IDOnlyGroup"] = "A"

id_sizes_no_name = (
    merged_df.loc[no_name]
             .groupby(["SourceGroup", "ID_num"], sort=False)["ID_num"]
             .transform("size")
)
dup_id_idx = merged_df.index[no_name & (id_sizes_no_name > 1)]
merged_df.loc[dup_id_idx, "isIDDup_NoName"] = 1
if len(dup_id_idx):
    rank_id_only = (
        merged_df.loc[dup_id_idx]
                 .sort_values(["SourceGroup", "ID_num", "CleanedFileName"])
                 .groupby(["SourceGroup", "ID_num"], sort=False)
                 .cumcount()
                 .map(idx_to_letters)
    )
    merged_df.loc[rank_id_only.index, "IDOnlyGroup"] = rank_id_only

# ========== 3) FinalGroup ==========
# 规则：
# - duplicated names → FinalGroup = NameGroup
# - duplicated ids→ FinalGroup = IDOnlyGroup
# - others → A
merged_df["FinalGroup"] = "A"
merged_df.loc[has_name & (merged_df["isNameDup"] == 1), "FinalGroup"] = merged_df["NameGroup"]
merged_df.loc[no_name & (merged_df["isIDDup_NoName"] == 1), "FinalGroup"] = merged_df["IDOnlyGroup"]

# merged_df.loc[merged_df["IsDup"] == 1, "FinalGroup"] = "A"

cols_show = [
    "CleanedFileName", "SourceGroup",
    "IsDup",
    "Subject", "Subject_norm",
    "ID", "ID_num",
    "isNameDup", "NameGroup",
    "isIDDup_Name", "NameIDGroup",
    "isIDDup_NoName", "IDOnlyGroup",
    "FinalGroup",
]
# display(merged_df[cols_show].head(50))
# display(merged_df[cols_show].tail(50))

merged_df.to_excel("all_regroups.xlsx", index=False)


Reading D:\SandmanGitCodes\files\final\group1.xlsx ...
Reading D:\SandmanGitCodes\files\final\group2.xlsx ...
Reading D:\SandmanGitCodes\files\final\group3.xlsx ...
Reading D:\SandmanGitCodes\files\final\group4.xlsx ...
Merged dataframe shape: (14914, 4)


C:\Users\gooda\AppData\Local\Temp\ipykernel_28096\860522918.py:122: FutureWarning: Operation between non boolean Series with different indexes will no longer return a boolean result in a future version. Cast both Series to object type to maintain the prior behavior.
  dup_name_id_idx = merged_df.index[mask_name_any & (name_id_sizes > 1)]
C:\Users\gooda\AppData\Local\Temp\ipykernel_28096\860522918.py:144: FutureWarning: Operation between non boolean Series with different indexes will no longer return a boolean result in a future version. Cast both Series to object type to maintain the prior behavior.
  dup_id_idx = merged_df.index[no_name & (id_sizes_no_name > 1)]


In [15]:
check_df = merged_df.rename(columns={
    "OutputPath": "Path",
    "CleanedFileName": "Name"
})[["FullPath", "Path", "Name", "SourceGroup", "FinalGroup"]]

check_df.head(20)

check_df["Name_length"] = check_df["Name"].str.len()

check_df["Name"] = check_df["Name"].astype(str).str.slice(0, 48)

check_df["Name"] = check_df["Name"].str.rstrip("-_")

check_df["Name_truncLength"] = check_df["Name"].str.len()

check_df.to_excel("check_name_len.xlsx", index=False)


In [ ]:
outdir = "groups_output"
os.makedirs(outdir, exist_ok=True)

export_df = check_df[["FullPath", "Path", "Name", "SourceGroup", "FinalGroup"]].copy()
export_df["Status"] = 0

for (sg, fg), subdf in export_df.groupby(["SourceGroup", "FinalGroup"]):
    subdf_out = subdf[["FullPath", "Path", "Name", "Status"]].copy()
    
    last_row = pd.DataFrame([["na", "na", "na", 0]], columns=subdf_out.columns)
    subdf_out = pd.concat([subdf_out, last_row], ignore_index=True)
    
    fname = f"{sg}_{fg}.xlsx"
    fpath = os.path.join(outdir, fname)
    
    subdf_out.to_excel(fpath, index=False)
    print(f"output: {fpath}, len={len(subdf_out)})")


output: groups_output\group1.xlsx_A.xlsx, len=3132)
output: groups_output\group1.xlsx_B.xlsx, len=246)
output: groups_output\group1.xlsx_C.xlsx, len=19)
output: groups_output\group1.xlsx_D.xlsx, len=3)
output: groups_output\group2.xlsx_A.xlsx, len=4601)
output: groups_output\group2.xlsx_B.xlsx, len=342)
output: groups_output\group2.xlsx_C.xlsx, len=20)
output: groups_output\group2.xlsx_D.xlsx, len=5)
output: groups_output\group2.xlsx_E.xlsx, len=3)
output: groups_output\group2.xlsx_F.xlsx, len=2)
output: groups_output\group3.xlsx_A.xlsx, len=3439)
output: groups_output\group3.xlsx_B.xlsx, len=123)
output: groups_output\group3.xlsx_C.xlsx, len=14)
output: groups_output\group3.xlsx_D.xlsx, len=4)
output: groups_output\group3.xlsx_E.xlsx, len=2)
output: groups_output\group4.xlsx_A.xlsx, len=2722)
output: groups_output\group4.xlsx_B.xlsx, len=229)
output: groups_output\group4.xlsx_C.xlsx, len=20)
output: groups_output\group4.xlsx_D.xlsx, len=5)
output: groups_output\group4.xlsx_E.xlsx, len

In [18]:
pivot_summary = (
    check_df.pivot_table(
        index="SourceGroup",
        columns="FinalGroup",
        values="Path",
        aggfunc="count",
        fill_value=0
    )
)

pivot_summary["Total"] = pivot_summary.sum(axis=1)

pivot_summary = pivot_summary.reset_index()

grp_info = {
    "group1.xlsx": ("7&9&14&16",  "TOSHIBA EXT"),
    "group2.xlsx": ("2&10&11&12&13", "SAMSUNG"),
    "group3.xlsx": ("1&3&8&15",   "KP Mirror"),
    "group4.xlsx": ("4&5&6",      "SandmanFiles"),
}

pivot_summary["Partitions"] = pivot_summary["SourceGroup"].map(lambda g: grp_info.get(g, ("", ""))[0])
pivot_summary["SSD"]        = pivot_summary["SourceGroup"].map(lambda g: grp_info.get(g, ("", ""))[1])

cols = list(pivot_summary.columns)
new_order = ["SourceGroup", "Partitions", "SSD"] + [c for c in cols if c not in {"SourceGroup","Partitions","SSD"}]
pivot_summary = pivot_summary[new_order]

pivot_summary.to_excel("groups_pivot_summary_with_partitions.xlsx", index=False)

display(pivot_summary.head(20))


FinalGroup,SourceGroup,Partitions,SSD,A,B,C,D,E,F,Total
0,group1.xlsx,7&9&14&16,TOSHIBA EXT,3131,245,18,2,0,0,3396
1,group2.xlsx,2&10&11&12&13,SAMSUNG,4600,341,19,4,2,1,4967
2,group3.xlsx,1&3&8&15,KP Mirror,3438,122,13,3,1,0,3577
3,group4.xlsx,4&5&6,SandmanFiles,2721,228,19,4,1,1,2974
